<a href="https://colab.research.google.com/github/sanzhbakh/data_mining/blob/master/gcn_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from copy import deepcopy

MAX_ROWS = 3
MAX_COLS = 3
MAX_DEPTH = 3
MAX_AGENTS = 2



In [0]:
def check_symmetric(a, tol=1e-8):
    return np.allclose(a, a.T, atol=tol)
        

In [0]:
class Graph():
    def __init__(self):
        self.max_nodes = MAX_ROWS * MAX_COLS * MAX_DEPTH
        self.max_agents = MAX_AGENTS
        self.max_depth = MAX_DEPTH
        self.depth_matrix = np.array([
            [2, 3],
            [0, 1]])
        self.rows, self.cols = self.depth_matrix.shape
        self.n_nodes = self.rows * self.cols * self.max_depth
        
        
    def reformat_depth_matrix(self):
        temp_matrix = np.zeros([self.rows * self.cols, self.max_depth])
        temp_3d_matrix = np.zeros([self.rows, self.cols, self.max_depth])
        
        
        for row in range(self.rows):
            for col in range(self.cols):
                depth = self.depth_matrix[row, col]
                temp_3d_matrix[row, col, :depth] = 1
       
        self.soil_3d = np.copy(temp_3d_matrix)
        
        
        idx = 0
        for row in range(self.rows):
            for col in range(self.cols):
                depth = self.depth_matrix[row, col]
                temp_matrix[idx, :depth].fill(1)
                idx += 1
#         temp_matrix = np.flip(temp_matrix, 1)
        self.depth_matrix_binary_2D = np.copy(temp_matrix)
        
    def get_surface(self):
        surface_cell_indices = []
        
        for i in range(self.depth_matrix_binary_2D.shape[0]):
            if self.depth_matrix_binary_2D[i,0] == 1:
                surface_cell_indices.append(i * self.max_depth)
        
        print('surface indices:', surface_cell_indices)
            
        
        return surface_cell_indices
    
    
    def construct_binary_t2t(self):
        
        surface = self.get_surface()
        
        self.binary_t2t = np.zeros([self.n_nodes, self.n_nodes])
#         print('binary t2t shape: ', self.binary_t2t.shape)
        
        # connect surface nodes
        for i in surface:
            for j in surface:
                self.binary_t2t[i, j] = 1
                
        # connect underground nodes to surface node
        for r in range(self.rows * self.cols):
            
            if self.depth_matrix_binary_2D[r, 0] == 0: # if surface is zero -> continue
                continue

            for j in range(1, self.max_depth):                
                print(r, j)
                if self.depth_matrix_binary_2D[r, j] == 1:
                    ug_node_index = r * self.max_depth + j
                    surface_node_index = ug_node_index - j
                    self.binary_t2t[ug_node_index, ug_node_index] = 1
                    self.binary_t2t[ug_node_index, surface_node_index] = 1
                    self.binary_t2t[surface_node_index, ug_node_index] = 1                   
                
       
        
#         print(self.binary_t2t)

#         print(check_symmetric(self.binary_t2t))
    
    def _get_2d_coords(self, node_index):
        row, col = node_index // self.max_depth, node_index % self.max_depth        
        return row, col
    
    def construct_t2t(self):      
        
        r = np.arange(self.rows)
        c = np.arange(self.cols)
        depth = np.arange(self.max_depth)
        rv, cv, dv = np.meshgrid(r, c, depth, sparse=False, indexing='ij')
        
        coords = np.transpose(np.vstack([rv.reshape([1, -1]), cv.reshape([1, -1]), dv.reshape([1, -1])]))
        
        self.t2t = np.zeros([self.n_nodes, self.n_nodes, 3])
        
        
        for node_i in range(coords.shape[0]):
            for node_j in range(coords.shape[0]):
                self.t2t[node_i, node_j, :] = abs((coords[node_i] - coords[node_j]))
                
                
        print('coords: ')
        print(coords)
        
        print('t2t: ')
        print(self.t2t[0, 8])
                
        # connect underground nodes to surface node
#         for i in range(n_nodes):
#             for j in range(n_nodes):
#                 if i == 0 or j == 0:
#                     continue
                
#                 self.t2t[i, j] = 
        
#         print(self.binary_t2t)

#         print(check_symmetric(self.binary_t2t))
        
        


In [75]:
graph = Graph()

print('depth_matrix:')
print(graph.depth_matrix)

graph.reformat_depth_matrix()


print('depth_matrix_binary_2D:')
print(graph.depth_matrix_binary_2D)
graph.construct_binary_t2t()
graph.construct_t2t()

depth_matrix:
[[2 3]
 [0 1]]
depth_matrix_binary_2D:
[[1. 1. 0.]
 [1. 1. 1.]
 [0. 0. 0.]
 [1. 0. 0.]]
surface indices: [0, 3, 9]
0 1
0 2
1 1
1 2
3 1
3 2
coords: 
[[0 0 0]
 [0 0 1]
 [0 0 2]
 [0 1 0]
 [0 1 1]
 [0 1 2]
 [1 0 0]
 [1 0 1]
 [1 0 2]
 [1 1 0]
 [1 1 1]
 [1 1 2]]
t2t: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]


In [85]:
graph.t2t[:, 1, 7]

array([1., 0., 0.])

In [70]:


binary_t2t = graph.depth_matrix_reformatted.reshape(-1)
binary_t2t

array([1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0.])

In [61]:
graph.depth_matrix_reformatted[:,-1]

array([1., 1., 1., 0.])

In [11]:
I = np.matrix(np.eye(A.shape[0]))
I

matrix([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [128]:
7 // 3

2

In [13]:
A_hat * X

matrix([[ 1., -1.],
        [ 6., -6.],
        [ 3., -3.],
        [ 5., -5.]])

In [19]:
# Normalizing feature representations
D = np.array(np.sum(A, axis=0))[0]
D = np.matrix(np.diag(D))
D_hat = np.array(np.sum(A_hat, axis=0))[0]
D_hat = np.matrix(np.diag(D_hat))
D_hat

matrix([[2., 0., 0., 0.],
        [0., 3., 0., 0.],
        [0., 0., 3., 0.],
        [0., 0., 0., 2.]])

In [24]:
W = np.matrix([
    [1, -1],
    [-1, 1]
])

print("notice that rows are summing up to 1. i.e. normalized")
D_hat**-1 * A_hat * X


notice that rows are summing up to 1. i.e. normalized


matrix([[ 0.5, -0.5],
        [ 2. , -2. ],
        [ 1. , -1. ],
        [ 2.5, -2.5]])

In [41]:
# Adding activation function
# We choose to preserve the dimensionality of the feature representations and apply the ReLU activation function.
f = D_hat**-1 * A_hat * X * W
print("f: \n", f)

f[f < 0] = 0
print("relu(f): \n", f)

f: 
 [[ 1. -1.]
 [ 4. -4.]
 [ 2. -2.]
 [ 5. -5.]]
relu(f): 
 [[1. 0.]
 [4. 0.]
 [2. 0.]
 [5. 0.]]


In [0]:
def relu(x):
    x[x<0] = 0
    return x

In [34]:

z

array([[-0.39335723, -0.86341238,  0.65038501],
       [-0.62958243,  0.68392315,  0.1753025 ],
       [ 0.75673649, -0.50031745,  0.9527737 ]])

In [33]:
relu(z)

array([[0.        , 0.        , 0.        ],
       [0.        , 0.24525935, 0.08134589],
       [0.28572315, 0.71689504, 0.30318565]])

In [39]:
z = np.random.uniform(-1, 1, (3,3))
print(z)
z[z < 0] = 0.0
z

[[-0.74361135 -0.10117558  0.35550908]
 [-0.43625849 -0.52675523  0.48083821]
 [-0.04531706  0.576386    0.97600195]]


array([[0.        , 0.        , 0.35550908],
       [0.        , 0.        , 0.48083821],
       [0.        , 0.576386  , 0.97600195]])

In [112]:
for x in range(1, 3):
    print(x)

1
2


In [0]:
r = np.arange(3)
c = np.arange(3)
depth = np.arange(3)
rv, cv, dv = np.meshgrid(r, c, depth, sparse=False, indexing='ij')

In [51]:
rv.reshape([1, -1])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2]])

In [48]:
cv.reshape(-1)

array([0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 1,
       1, 1, 2, 2, 2])

In [49]:
dv.reshape(-1)

array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0,
       1, 2, 0, 1, 2])

In [68]:
coords = np.transpose(np.vstack([rv.reshape([1, -1]), cv.reshape([1, -1]), dv.reshape([1, -1])]))
abs(coords[3] - coords[5])

array([0, 0, 2])

In [65]:
np.zeros([2,3,3])

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])